
<center> <h5> Badania Operacyjne </h5> <h1> Zagadnienie przydziału – algorytm
(metoda) węgierska </h1> </center>
<div style="text-align: right"> <i> Łukasz Faruga </i></div> 
<div style="text-align: right"> <i> Adam Filapek </i></div> 
<div style="text-align: right"> <i> Andrzej Neścior  </i></div> 

## Zadanie 1

Implementacja algortymu

In [2]:
import copy
import numpy as np
from typing import List, Tuple, Optional


def get_independent_zeros(matrix_after_reduction: np.ndarray) -> Optional[List[Tuple[int, int]]]:
    """
    :param matrix_after_reduction: kwadratowa macierz liczb nieujemnych, zredukowana
    :return: lista indeksów zer niezależnych lub -None-, gdy brak rozwiązania
    """

    points: List[Tuple[int, int]] = []
    _m: np.ndarray = copy.deepcopy(matrix_after_reduction)

    for _ in range(_m.shape[0]):
        # zlicz zera w każdym wierszu
        zeros: np.ndarray = np.count_nonzero(_m == 0, axis=1)

        # test czy w iteracji znaleziono już zero niezależne
        zero_found: bool = False

        # jeśli w wierszu jest jedno zero, to jest niezależne
        temp: np.ndarray = np.nonzero(zeros == 1)[0]
        if temp.size > 0:
            # wyznaczenie współrzędnych
            row_idx: int = temp[0]
            col_idx: int = np.where(_m[row_idx, :] == 0)[0][0]
            points.append((row_idx, col_idx))

            # usunięcie wiersza i kolumny z macierzy
            _m[:, col_idx] = _m[:, col_idx] + np.inf
            _m[row_idx, :] = _m[row_idx, :] + np.inf

            zero_found = True

        # jeśli w wierszu jest więcej niż jedno zero, to wybierz pierwsze
        if not zero_found:
            temp: np.ndarray = np.nonzero(zeros > 1)[0]
            if temp.size > 0:
                # wyznaczenie współrzędnych
                row_idx: int = temp[0]
                col_idx: int = np.where(_m[row_idx, :] == 0)[0][0]
                points.append((row_idx, col_idx))

                # usunięcie wiersza i kolumny z macierzy
                _m[:, col_idx] = _m[:, col_idx] + np.inf
                _m[row_idx, :] = _m[row_idx, :] + np.inf

                zero_found = True

        # jeśli brak zer w macierzy, to zakończ
        if not zero_found:
            break

    return points


def min_lines(m, zeros_dependant):
    X, _ = m.shape
    zeros = m == 0
    zeros = zeros.astype(int)
    # indeksy oznaczonych kolumn/wierszy
    marked_rows = set()
    marked_cols = set()
    for i, j in zeros_dependant:
        zeros[i, j] = -1

    for idx, row in enumerate(zeros):
        for el in row:
            if el == -1:  # zero niezależne
                break
        else:
            marked_rows.add(idx)

    while True:
        changes = False

        for row in range(X):
            for col in range(X):
                # zero zależne i oznaczony wiersz
                if zeros[row, col] == 1 and row in marked_rows and col not in marked_cols:
                    marked_cols.add(col)
                    changes = True

        for row in range(X):
            for col in range(X):
                # zero niezależne i oznaczona kolumna
                if zeros[row, col] == -1 and col in marked_cols and row not in marked_rows:
                    marked_rows.add(row)
                    changes = True

        if not changes:
            break

    all_rows = set([row_idx for row_idx in range(X)])

    return all_rows.difference(marked_rows), marked_cols


def matrix_reduction(original_matrix: np.ndarray) -> (np.ndarray, np.ndarray, int):
    cost = 0
    buffor_matrix = copy.deepcopy(original_matrix)

    for row in buffor_matrix:
        if not np.any(row == 0):
            minimum = np.min(row)
            cost += minimum
            row -= minimum

    buffor_matrix = buffor_matrix.T

    for row in buffor_matrix:
        if not np.any(row == 0):
            minimum = np.min(row)
            cost += minimum
            row -= minimum

    buffor_matrix = buffor_matrix.T

    return buffor_matrix, cost


def shift_zeros(matrix_after_reduction: np.ndarray, marked_rows: set, marked_cols: set):
    X, _ = matrix_after_reduction.shape
    cost = 0
    #tworze macierz 0 i 1 gdzie 0 to linia
    zeros = np.ones((X,X)).astype(int)
    for row in marked_rows:
        zeros[row,:] = 0
    for col in marked_cols:
        zeros[:,col] = 0

    buffor_matrix = np.multiply(zeros, matrix_after_reduction)
    # szukam wartości minimalnej w macierzy
    minimal = np.min(buffor_matrix[np.nonzero(buffor_matrix)])
    # szukam punktów przecięcia linii
    crossed_points = []
    if len(marked_rows) == 0 or len(marked_cols) == 0:
        pass
    else:
        for x in marked_rows:
            for y in marked_cols:
                crossed_points.append((x, y))

    # odejmuję wartość minimalną od odpowiednich elementów
    for row in range(X):
        for col in range(X):
            if zeros[row, col]-1 == 0:
                matrix_after_reduction[row, col] -= minimal

    # dodaje wartość minimalną do odpowiednich elementów
    for row, col in crossed_points:
        matrix_after_reduction[row, col] += minimal

    cost = len(crossed_points)*minimal

    return matrix_after_reduction, cost


## Zadanie 2
Demonstracja działania algorytmu dla losowo wygenerowanej macierzy.

In [6]:
def main():
    matrix = np.random.randint(10, size=(6, 6))
    print("Macierz oryginalna: \n", matrix)
    reduced_matrix, cost = matrix_reduction(matrix)
    print("Macierz zredukowana: \n", reduced_matrix)
    for x in range(40):
        result = get_independent_zeros(reduced_matrix)
        if len(result) != reduced_matrix.shape[0]:
            reduced_matrix, cost_buff = shift_zeros(reduced_matrix, *min_lines(reduced_matrix, result))
            cost += cost_buff
            print("Zamiana zer macierzy: \n", reduced_matrix)
        elif len(result) == reduced_matrix.shape[0]:
            print("Rozwiązanie końcowe: ", result, '\nOptymalny koszt przydziału: ', cost)
            break

if __name__ == '__main__':
    main()


Macierz oryginalna: 
 [[7 8 2 6 0 5]
 [3 9 3 6 3 8]
 [7 3 2 2 6 0]
 [8 8 2 7 5 3]
 [1 5 5 6 5 3]
 [2 4 8 5 1 9]]
Macierz zredukowana: 
 [[7 5 2 4 0 5]
 [0 3 0 1 0 5]
 [7 0 2 0 6 0]
 [6 3 0 3 3 1]
 [0 1 4 3 4 2]
 [1 0 7 2 0 8]]
Zamiana zer macierzy: 
 [[7 4 2 3 0 4]
 [0 2 0 0 0 4]
 [8 0 3 0 7 0]
 [6 2 0 2 3 0]
 [0 0 4 2 4 1]
 [2 0 8 2 1 8]]
Rozwiązanie końcowe:  [(0, 4), (5, 1), (4, 0), (1, 2), (3, 5), (2, 3)] 
Optymalny koszt przydziału:  18


<div style="page-break-after: always;"></div>

## Zadanie 3

* Czy wynik redukcji jest zależny od kolejności (wiersze-kolumny/ kolumny-wiersze) – uzyskamy zawsze tą samą macierz / sumaryczną wielkość redukcji?
    * Wynik redukcji nie zależy od tego czy najpierw zredukowane zostaną wiersze czy kolumny, ponieważ i-ty wiersz jest powiązany z j-tą kolumną przez wartość w komórce (i, j) macierzy, co skutkuje tym, że redukcja i-tego wiersza wpływa na redukcję j-tej kolumny i na odwrót.
* Jak jest możliwa minimalna / maksymalna liczba zer niezależnych w macierzy zredukowanej?
    * Minimalna liczba zer niezależnych w macierzy zredukowanej o wymiarze n to 1, przykładem jest macierz z pierwszym wierszem i pierwszą kolumną wypełnionymi zerami, natomiast pozostałe wartości są różne od zera.
    * Maksymalna liczba zer niezależnych w macierzy zredukowanej o wymiarze n to n, przykładem jest macierz z zerami na przekątnej, a wartościami różnymi od zera poza nią.
* Czy wykreślanie zer macierzy jest prawidłowa (zawsze) jeśli będziemy wykreślać kolejno linie (wiesz/kolumna) z największą liczbą nieskreślonych zer?
    * Podejście, w którym wykreślamy kolejno wiersze bądź kolumny z największą liczbą nieskreślonych zer jest prawidłowe. 
* Jak się ma minimalna liczba linii (wykreślająca zera) i liczba (maksymalna) zer niezależnych?
    * Minimalna liczba linii wykreślających zera oraz maksymalna liczba zer niezależnych są sobie równe.
* Czy procedura zwiększająca liczbę zer niezależnych zawsze jest skuteczna / o ile
może zmienić się ich liczba?
    * Procedura ta zawsze jest skuteczna, ponieważ w każdej iteracji dokładane jest co najmniej jedno zero. Powtarzanie tej procedury zawsze da oczekiwany wynik, gdyż dla każdej macierzy ten wynik istnieje. 
    * Liczba zer może się zwiększyć o nie więcej, niż różnica między obecną liczbą zer niezależnych a rozmiarem macierzy. 
